# **Integration of ERA5-Land Daily Weather Features for Fire Alerts (Google Earth Engine API)**


In [ ]:
import ee
import geemap
import pandas as pd
from tqdm import tqdm
from google.colab import files
from datetime import datetime

In [ ]:
# Initialize with your project ID
project_id = "ee-903660" #actual project ID
try:
    ee.Initialize(project=project_id)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=project_id)

In [ ]:
# Load your dataset
firealerts = pd.read_csv('/content/para_data2022_with_ndvi_evi.csv')

In [ ]:
# Function to extract weather data for a single row
def extract_weather_features(row):
    lat = row['latitude']
    lon = row['longitude']
    date_str = row['alert__date']
    point = ee.Geometry.Point([lon, lat])

    # Use the same date (daily values)
    date = ee.Date(date_str)

    # Load ERA5-Land Daily Aggregates for that day
    try:
        weather = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
            .filterDate(date, date.advance(1, 'day')) \
            .first()

        # Extract relevant bands
        bands = [
            'temperature_2m_max',
            'temperature_2m_min',
            'total_precipitation_sum',
            'u_component_of_wind_10m',
            'v_component_of_wind_10m'
        ]

        values = weather.select(bands).reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        return pd.Series([
            values.get('temperature_2m_max'),
            values.get('temperature_2m_min'),
            values.get('total_precipitation_sum'),
            values.get('u_component_of_wind_10m'),
            values.get('v_component_of_wind_10m')
        ])

    except Exception as e:
        print(f"Error at ({lat}, {lon}, {date_str}): {e}")
        return pd.Series([None] * 5)

# Rename output columns for clarity
weather_column_names = [
    'temp_max',
    'temp_min',
    'precipitation',
    'wind_u_10m',
    'wind_v_10m'
]

In [ ]:
batch_size = 1000
batches = []
import os
os.makedirs("para_batches", exist_ok=True)

for i, start in enumerate(range(0, len(firealerts), batch_size)):
    end = start + batch_size
    batch = firealerts.iloc[start:end]

    # Apply the function to the batch
    weather_features_batch = batch.apply(extract_weather_features, axis=1)
    weather_features_batch.columns = weather_column_names

    # Merge with the original batch
    merged_batch = pd.concat([batch, weather_features_batch], axis=1)
    batches.append(merged_batch)

    # Save individual batch to CSV
    batch_filename = f"para_batches/para22_batch_{i}.csv"
    merged_batch.to_csv(batch_filename, index=False)
    print(f"Saved {batch_filename}")

# Save full concatenated data
firealerts_with_weather = pd.concat(batches, axis=0)
firealerts_with_weather.to_csv("para_data2022_with_ndvi&weather.csv", index=False)
print("All batches processed and final file saved.")

Saved para_batches/para22_batch_0.csv
Saved para_batches/para22_batch_1.csv
Saved para_batches/para22_batch_2.csv


Error at (-8.62599, -54.21996, 2022-07-25): Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access


Error at (-7.15272, -55.01887, 2022-07-06): Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access


Error at (-6.2626, -52.63503, 2022-07-21): Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access
Saved para_batches/para22_batch_3.csv


Error at (-5.86335, -52.07313, 2022-07-20): Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access
Saved para_batches/para22_batch_4.csv
Saved para_batches/para22_batch_5.csv
Saved para_batches/para22_batch_6.csv
Saved para_batches/para22_batch_7.csv
Saved para_batches/para22_batch_8.csv
Saved para_batches/para22_batch_9.csv
Saved para_batches/para22_batch_10.csv
Saved para_batches/para22_batch_11.csv
Saved para_batches/para22_batch_12.csv
Saved para_batches/para22_batch_13.csv
Saved para_batches/para22_batch_14.csv
Saved para_batches/para22_batch_15.csv
Saved para_batches/para22_batch_16.csv
Saved para_batches/para22_batch_17.csv
Saved para_batches/para22_batch_18.csv
Saved para_batches/para22_batch_19.csv
Saved para_batches/para22_batch_20.csv
Saved para_batches/para22_batch_21.csv
Saved para_batches/para22_batch_22.csv
Saved para_batches/para22_batch_23.csv
Saved para_batches/para22_batch_24.csv
Saved para_batches/